In [1]:
!pip install gradio pandas numpy matplotlib seaborn plotly

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import re
import random
from collections import Counter, defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import threading
import time

# Enhanced Government Platform with IBM Granite AI Integration
class GovernmentPlatform:
    def __init__(self):
        self.supported_languages = ["English", "Hindi", "Telugu", "Tamil", "Bengali", "Marathi"]
        self.query_categories = ["Healthcare", "Education", "Infrastructure", "Employment", "Welfare", "Legal", "Other"]
        self.setup_theme()
        self.init_mock_data()
        self.init_granite_model()

    def init_granite_model(self):
        """Initialize IBM Granite 3.3 2B Instruct model"""
        try:
            print("Loading IBM Granite 3.3 2B Instruct model...")
            self.model_name = "ibm-granite/granite-3.3-2b-instruct"

            # Load tokenizer and model
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                trust_remote_code=True
            )

            # Create text generation pipeline
            self.text_generator = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.1
            )

            # Government-specific context for better responses
            self.government_context = """
You are an AI assistant for a Government Citizen Engagement Platform. You help citizens with:
- Government services and procedures
- Document applications (passport, ration card, birth certificate, etc.)
- Welfare schemes and benefits
- Legal procedures and rights
- Healthcare services
- Education policies
- Infrastructure issues
- Employment opportunities

Provide helpful, accurate, and official information. Always be polite and professional.
If you don't know specific details, guide users to appropriate government offices or websites.
"""

            self.model_loaded = True
            print("IBM Granite model loaded successfully!")

        except Exception as e:
            print(f"Error loading Granite model: {e}")
            print("Falling back to mock responses...")
            self.model_loaded = False
            self.setup_fallback_responses()

    def setup_fallback_responses(self):
        """Setup fallback responses if model fails to load"""
        self.fallback_responses = {
            "passport": "For passport services, visit your nearest passport office or apply online at passportindia.gov.in. Required documents include proof of identity, address, and date of birth.",
            "ration card": "Ration card applications can be submitted at your local PDS office or online through the state portal. You'll need identity proof, address proof, and income certificate.",
            "birth certificate": "Birth certificates can be obtained from the municipal corporation or registrar office where the birth was registered. Online applications are available through the respective state portals.",
            "pension": "Pension schemes are available for senior citizens, widows, and disabled persons. Visit your nearest social welfare office with identity proof, age proof, and bank details.",
            "healthcare": "Government healthcare services are available through primary health centers, community health centers, and government hospitals. AYUSHMAN Bharat provides health insurance coverage.",
            "education": "Government education schemes include scholarships, mid-day meals, and free textbooks. Visit your nearest education office or check the respective state education portal.",
            "employment": "Employment opportunities are available through MGNREGA, skill development programs, and government job portals. Register on the National Career Service portal.",
            "legal": "For legal aid, contact the District Legal Services Authority or visit the nearest court. Free legal aid is available for eligible citizens.",
            "infrastructure": "Infrastructure issues can be reported to the municipal corporation, panchayat, or through citizen grievance portals. Most states have dedicated complaint systems."
        }

    def setup_theme(self):
        """Setup government-themed interface"""
        self.theme = gr.themes.Soft(
            primary_hue="blue",
            secondary_hue="indigo",
            neutral_hue="slate"
        ).set(
            body_background_fill="linear-gradient(135deg, #667eea 0%, #764ba2 100%)",
            block_background_fill="white",
            block_border_color="#e5e7eb"
        )

    def init_mock_data(self):
        """Initialize mock data for demonstration"""
        self.citizens_data = pd.DataFrame({
            'citizen_id': range(1, 1001),
            'queries_count': np.random.poisson(3, 1000),
            'satisfaction_score': np.random.uniform(1, 5, 1000),
            'language': np.random.choice(self.supported_languages, 1000),
            'category': np.random.choice(self.query_categories, 1000),
            'resolution_time': np.random.exponential(2, 1000)
        })

        self.queries_data = []
        for i in range(100):
            self.queries_data.append({
                'id': f"Q{i+1:03d}",
                'citizen_id': f"C{random.randint(1, 1000):04d}",
                'query': f"Sample query about {random.choice(self.query_categories).lower()}",
                'category': random.choice(self.query_categories),
                'sentiment': random.choice(['Positive', 'Neutral', 'Negative']),
                'status': random.choice(['Open', 'In Progress', 'Resolved', 'Closed']),
                'priority': random.choice(['Low', 'Medium', 'High', 'Critical']),
                'timestamp': datetime.now() - timedelta(days=random.randint(0, 30))
            })

    def analyze_sentiment(self, text):
        """Analyze sentiment of the input text"""
        # Simple sentiment analysis
        positive_words = ['good', 'great', 'excellent', 'satisfied', 'happy', 'pleased', 'wonderful', 'amazing', 'perfect', 'helpful']
        negative_words = ['bad', 'terrible', 'awful', 'unsatisfied', 'angry', 'frustrated', 'horrible', 'worst', 'disappointed', 'annoyed']

        text_lower = text.lower()
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)

        if positive_count > negative_count:
            return "Positive"
        elif negative_count > positive_count:
            return "Negative"
        else:
            return "Neutral"

    def generate_granite_response(self, user_message, chat_history):
        """Generate response using IBM Granite model"""
        try:
            # Prepare the conversation context
            conversation_context = self.government_context + "\n\n"

            # Add chat history for context
            for human_msg, ai_msg in chat_history[-3:]:  # Last 3 exchanges for context
                conversation_context += f"User: {human_msg}\nAssistant: {ai_msg}\n\n"

            # Add current user message
            prompt = f"{conversation_context}User: {user_message}\nAssistant:"

            # Generate response
            response = self.text_generator(
                prompt,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=self.tokenizer.eos_token_id
            )

            # Extract the generated text
            generated_text = response[0]['generated_text']

            # Extract only the assistant's response
            assistant_response = generated_text.split("Assistant:")[-1].strip()

            # Clean up the response
            assistant_response = assistant_response.split("User:")[0].strip()
            assistant_response = assistant_response.split("\n\n")[0].strip()

            # Ensure the response is not empty
            if not assistant_response or len(assistant_response) < 10:
                return self.get_fallback_response(user_message)

            return assistant_response

        except Exception as e:
            print(f"Error generating Granite response: {e}")
            return self.get_fallback_response(user_message)

    def get_fallback_response(self, message):
        """Get fallback response when AI model fails"""
        message_lower = message.lower()

        # Check for keywords in fallback responses
        for keyword, response in self.fallback_responses.items():
            if keyword in message_lower:
                return response

        # Default response
        return "I'm here to help you with government services and procedures. Could you please provide more specific details about what you need assistance with? I can help with passport applications, ration cards, birth certificates, pension schemes, healthcare services, education policies, employment opportunities, legal aid, and infrastructure issues."

    def chatbot_response(self, message, chat_history):
        """Enhanced AI Chatbot with IBM Granite integration"""
        if not message.strip():
            return "", chat_history

        # Generate response using Granite model or fallback
        if self.model_loaded:
            try:
                response = self.generate_granite_response(message, chat_history)
            except Exception as e:
                print(f"Granite model error: {e}")
                response = self.get_fallback_response(message)
        else:
            response = self.get_fallback_response(message)

        # Analyze sentiment
        sentiment = self.analyze_sentiment(message)

        # Add sentiment analysis to response
        response_with_sentiment = f"{response}\n\n📊 *Sentiment Analysis: {sentiment}*"

        # Add to chat history
        chat_history.append([message, response_with_sentiment])

        return "", chat_history

    def process_citizen_query(self, query_text, language, voice_input, category):
        """Process citizen queries with AI enhancement"""
        if not query_text and not voice_input:
            return "Please provide a query either as text or voice input."

        processed_query = query_text if query_text else "[Voice input transcribed]"

        # Use AI to categorize and analyze the query if available
        if self.model_loaded and query_text:
            try:
                # Generate AI analysis of the query
                analysis_prompt = f"""
                Analyze this citizen query and provide:
                1. Category classification
                2. Priority level
                3. Estimated resolution time
                4. Required documents/information

                Query: {query_text}
                Category: {category}
                """

                ai_analysis = self.text_generator(
                    analysis_prompt,
                    max_new_tokens=150,
                    temperature=0.3
                )[0]['generated_text'].split("Query:")[-1].strip()

            except:
                ai_analysis = "Standard processing applied"
        else:
            ai_analysis = "Standard processing applied"

        response = {
            "query_id": f"Q{random.randint(1000, 9999)}",
            "original_language": language,
            "processed_query": processed_query,
            "category": category,
            "ai_analysis": ai_analysis,
            "estimated_resolution_time": f"{random.randint(1, 7)} days",
            "status": "Received and being processed with AI assistance",
            "reference_number": f"REF{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "ai_model": "IBM Granite 3.3-2B" if self.model_loaded else "Fallback System"
        }

        return json.dumps(response, indent=2)

    def detect_misinformation(self, text_content, source_url):
        """Enhanced misinformation detection with AI"""
        if not text_content:
            return "Please provide content to analyze"

        # Basic keyword-based risk assessment
        risk_keywords = ['fake', 'hoax', 'conspiracy', 'unverified', 'rumor', 'false', 'misleading']
        risk_score = sum(1 for keyword in risk_keywords if keyword.lower() in text_content.lower())

        # AI-enhanced analysis if model is available
        if self.model_loaded:
            try:
                verification_prompt = f"""
                Analyze this content for potential misinformation. Consider:
                1. Factual accuracy
                2. Source credibility indicators
                3. Emotional language usage
                4. Verification recommendations

                Content: {text_content[:500]}...
                """

                ai_verification = self.text_generator(
                    verification_prompt,
                    max_new_tokens=200,
                    temperature=0.3
                )[0]['generated_text'].split("Content:")[-1].strip()

            except:
                ai_verification = "AI analysis unavailable"
        else:
            ai_verification = "AI analysis unavailable"

        credibility_score = max(0, 100 - (risk_score * 15) - random.randint(0, 20))

        analysis = {
            "content_length": len(text_content),
            "risk_keywords_found": risk_score,
            "credibility_score": f"{credibility_score}%",
            "classification": "High Risk" if credibility_score < 40 else "Medium Risk" if credibility_score < 70 else "Low Risk",
            "ai_analysis": ai_verification,
            "recommendations": [
                "Verify with official government sources",
                "Check multiple authoritative references",
                "Cross-reference with government portals",
                "Consult official press releases"
            ] if credibility_score < 70 else ["Content appears credible based on analysis"],
            "analysis_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "ai_model_used": "IBM Granite 3.3-2B" if self.model_loaded else "Rule-based system"
        }

        return json.dumps(analysis, indent=2)

    # Keep all other methods from the original class unchanged
    def get_summary(self):
        return {
            'total_queries': len(self.citizens_data['queries_count']),
            'avg_satisfaction': round(np.mean(self.citizens_data['satisfaction_score']), 2),
            'avg_resolution_time': round(np.mean(self.citizens_data['resolution_time']), 2)
        }

    def filter_by_language(self, lang):
        mask = np.array(self.citizens_data['language']) == lang
        return {key: np.array(val)[mask] for key, val in self.citizens_data.items()}

    def generate_admin_dashboard(self):
        """Generate real-time dashboard analytics"""
        dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
        query_volume = np.random.poisson(50, len(dates))

        fig1 = px.line(x=dates, y=query_volume, title="Daily Query Volume")
        fig1.update_layout(xaxis_title="Date", yaxis_title="Number of Queries")

        category_counts = Counter(self.citizens_data['category'])
        fig2 = px.pie(values=list(category_counts.values()),
                     names=list(category_counts.keys()),
                     title="Query Distribution by Category")

        fig3 = px.histogram(self.citizens_data, x='satisfaction_score',
                           title="Citizen Satisfaction Distribution",
                           nbins=20)

        language_counts = Counter(self.citizens_data['language'])
        fig4 = px.bar(x=list(language_counts.keys()),
                     y=list(language_counts.values()),
                     title="Platform Usage by Language")

        return fig1, fig2, fig3, fig4

    def staff_query_management(self, staff_role, filter_category, filter_status):
        """Government staff query management interface"""
        filtered_queries = self.queries_data.copy()

        if filter_category != "All":
            filtered_queries = [q for q in filtered_queries if q['category'] == filter_category]

        if filter_status != "All":
            filtered_queries = [q for q in filtered_queries if q['status'] == filter_status]

        df = pd.DataFrame(filtered_queries)

        summary = {
            "total_queries": len(filtered_queries),
            "open_queries": len([q for q in filtered_queries if q['status'] == 'Open']),
            "resolved_queries": len([q for q in filtered_queries if q['status'] == 'Resolved']),
            "average_resolution_time": f"{np.mean([q['timestamp'].day for q in filtered_queries]):.1f} days",
            "staff_role": staff_role,
            "ai_assistance": "IBM Granite 3.3-2B Active" if self.model_loaded else "Basic System"
        }

        return df.head(20), json.dumps(summary, indent=2)

    def generate_analytics_report(self, report_type, date_range):
        """Generate comprehensive analytics reports"""
        if report_type == "Performance Metrics":
            metrics = {
                "total_citizens_served": len(self.citizens_data),
                "average_satisfaction_score": f"{self.citizens_data['satisfaction_score'].mean():.2f}/5.0",
                "total_queries_processed": len(self.queries_data),
                "resolution_rate": f"{random.randint(75, 95)}%",
                "average_response_time": f"{random.randint(2, 8)} hours",
                "most_popular_service": random.choice(self.query_categories),
                "peak_usage_time": "10:00 AM - 2:00 PM",
                "mobile_app_usage": f"{random.randint(60, 80)}%",
                "ai_model_status": "IBM Granite 3.3-2B Active" if self.model_loaded else "Fallback System"
            }

        elif report_type == "Trend Analysis":
            metrics = {
                "query_growth_rate": f"+{random.randint(5, 25)}% monthly",
                "satisfaction_trend": "Improving (+0.3 points)",
                "digital_adoption_rate": f"{random.randint(70, 90)}%",
                "language_preference_shift": "Increasing vernacular usage",
                "service_demand_forecast": "Healthcare queries expected to increase by 15%",
                "seasonal_patterns": "Higher queries during festival seasons",
                "user_retention_rate": f"{random.randint(80, 95)}%",
                "ai_enhancement_impact": "25% improvement in response quality" if self.model_loaded else "Baseline performance"
            }

        else:  # Predictive Insights
            metrics = {
                "predicted_query_volume": f"{random.randint(1000, 5000)} next month",
                "resource_optimization": "Deploy 2 additional staff in Healthcare dept",
                "citizen_satisfaction_prediction": "Expected to reach 4.5/5.0",
                "service_expansion_recommendation": "Add multi-modal transport services",
                "infrastructure_needs": "Upgrade server capacity by 30%",
                "staff_training_priority": "AI-assisted query resolution",
                "budget_allocation_suggestion": "45% tech/AI, 30% staff, 25% infrastructure"
            }

        return json.dumps(metrics, indent=2)

    def send_notifications(self, notification_type, target_audience, message, channels):
        """Send notifications and alerts to citizens"""
        simulation = {
            "notification_id": f"NOT_{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "type": notification_type,
            "target_audience": target_audience,
            "message": message,
            "channels": channels,
            "estimated_reach": random.randint(1000, 50000),
            "delivery_status": "Scheduled for immediate delivery",
            "cost_estimate": f"₹{random.randint(500, 5000)}",
            "delivery_time": f"{random.randint(5, 30)} minutes",
            "success_rate_prediction": f"{random.randint(85, 98)}%"
        }

        return json.dumps(simulation, indent=2)

    def authenticate_user(self, username, role, access_level):
        """Simulate user authentication and role-based access"""
        if not username:
            return "Please provide username"

        auth_result = {
            "username": username,
            "role": role,
            "access_level": access_level,
            "login_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "session_id": f"SES_{random.randint(100000, 999999)}",
            "permissions": {
                "view_dashboard": role in ["Admin", "Manager", "Staff"],
                "manage_queries": role in ["Admin", "Manager", "Staff"],
                "access_analytics": role in ["Admin", "Manager"],
                "system_administration": role == "Admin",
                "citizen_data_access": access_level in ["Full", "Partial"],
                "ai_model_access": self.model_loaded
            },
            "last_login": (datetime.now() - timedelta(days=random.randint(1, 7))).strftime("%Y-%m-%d"),
            "failed_attempts": 0,
            "account_status": "Active",
            "ai_features": "IBM Granite 3.3-2B Enabled" if self.model_loaded else "Basic Features"
        }

        return json.dumps(auth_result, indent=2)

    def create_interface(self):
        """Create the comprehensive government platform interface"""

        with gr.Blocks(theme=self.theme, title="Government Citizen Engagement Platform") as interface:

            # Header
            gr.Markdown(f"""
            # 🏛️ Government Citizen Engagement Platform
            ### IBM Hackathon 2024 - Digital Governance Solution

            **AI Enhancement Status:** {'✅ IBM Granite 3.3-2B Active' if self.model_loaded else '⚠️ Fallback System Active'}

            Comprehensive platform for citizen services, query management, and government analytics
            """)

            with gr.Tabs():
                # Module 1: Citizen Interface
                with gr.TabItem("👥 Citizen Portal"):
                    gr.Markdown("### Submit Your Query - Voice & Multi-language Support")

                    with gr.Row():
                        with gr.Column():
                            citizen_query = gr.Textbox(
                                label="Your Query",
                                placeholder="Describe your issue or question...",
                                lines=4
                            )
                            language_select = gr.Dropdown(
                                choices=self.supported_languages,
                                label="Preferred Language",
                                value="English"
                            )
                            voice_input = gr.Audio(
                                label="Voice Input (Optional)",
                                type="filepath"
                            )
                            query_category = gr.Dropdown(
                                choices=self.query_categories,
                                label="Query Category",
                                value="Other"
                            )
                            submit_query_btn = gr.Button("📤 Submit Query", variant="primary")

                        with gr.Column():
                            query_response = gr.Textbox(
                                label="Query Processing Result",
                                lines=12,
                                show_copy_button=True
                            )

                # Module 2: Enhanced AI Chatbot
                with gr.TabItem("🤖 AI Assistant (IBM Granite)"):
                    gr.Markdown(f"""
                    ### AI-Powered Government Assistant
                    **Powered by:** {'IBM Granite 3.3-2B Instruct Model' if self.model_loaded else 'Fallback Response System'}

                    Get instant help with government services, procedures, and information.
                    """)

                    chatbot = gr.Chatbot(
                        label="Government AI Assistant",
                        height=400,
                        placeholder="Ask me about government services, procedures, or any questions you have!"
                    )
                    with gr.Row():
                        chat_input = gr.Textbox(
                            label="Ask me anything...",
                            placeholder="Type your question here (e.g., 'How to apply for passport?')",
                            scale=4
                        )
                        chat_submit = gr.Button("Send", variant="primary", scale=1)

                    gr.Examples(
                        examples=[
                            "How do I apply for a ration card?",
                            "What documents are needed for passport application?",
                            "Tell me about pension scheme eligibility",
                            "How to get a birth certificate?",
                            "What are the steps for starting a business?",
                            "How can I access healthcare services?",
                            "What employment opportunities are available?",
                            "How do I file a legal complaint?"
                        ],
                        inputs=chat_input
                    )

                # Module 3: Enhanced Misinformation Detection
                with gr.TabItem("🔍 AI Content Verification"):
                    gr.Markdown(f"""
                    ### AI-Powered Content Verification System
                    **Enhanced with:** {'IBM Granite 3.3-2B Analysis' if self.model_loaded else 'Rule-based Detection'}
                    """)

                    with gr.Row():
                        with gr.Column():
                            content_text = gr.Textbox(
                                label="Content to Verify",
                                placeholder="Paste the content you want to fact-check...",
                                lines=8
                            )
                            source_url = gr.Textbox(
                                label="Source URL (Optional)",
                                placeholder="https://example.com/article"
                            )
                            verify_btn = gr.Button("🔍 Verify Content", variant="secondary")

                        with gr.Column():
                            verification_result = gr.Textbox(
                                label="AI Verification Analysis",
                                lines=12,
                                show_copy_button=True
                            )

                # Keep all other tabs unchanged...
                # Module 4: Admin Dashboard
                with gr.TabItem("📊 Admin Dashboard"):
                    gr.Markdown("### Real-time Analytics & Insights")

                    dashboard_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")

                    with gr.Row():
                        query_volume_plot = gr.Plot(label="Query Volume Trends")
                        category_plot = gr.Plot(label="Category Distribution")

                    with gr.Row():
                        satisfaction_plot = gr.Plot(label="Satisfaction Scores")
                        language_plot = gr.Plot(label="Language Usage")

                # Module 5: Staff Portal
                with gr.TabItem("👨‍💼 Government Staff Portal"):
                    gr.Markdown("### Query Management & Resolution Tracking")

                    with gr.Row():
                        staff_role = gr.Dropdown(
                            choices=["Admin", "Manager", "Staff", "Specialist"],
                            label="Staff Role",
                            value="Staff"
                        )
                        filter_category = gr.Dropdown(
                            choices=["All"] + self.query_categories,
                            label="Filter by Category",
                            value="All"
                        )
                        filter_status = gr.Dropdown(
                            choices=["All", "Open", "In Progress", "Resolved", "Closed"],
                            label="Filter by Status",
                            value="All"
                        )
                        load_queries_btn = gr.Button("📋 Load Queries", variant="primary")

                    with gr.Row():
                        queries_table = gr.Dataframe(
                            label="Query Management Dashboard",
                            interactive=True
                        )
                        staff_summary = gr.Textbox(
                            label="Summary Statistics",
                            lines=10,
                            show_copy_button=True
                        )

                # Module 6: Analytics & Reporting
                with gr.TabItem("📈 Analytics & Reports"):
                    gr.Markdown("### Comprehensive Analytics & Predictive Insights")

                    with gr.Row():
                        report_type = gr.Radio(
                            choices=["Performance Metrics", "Trend Analysis", "Predictive Insights"],
                            label="Report Type",
                            value="Performance Metrics"
                        )
                        date_range = gr.Dropdown(
                            choices=["Last 7 days", "Last 30 days", "Last 3 months", "Last year"],
                            label="Date Range",
                            value="Last 30 days"
                        )
                        generate_report_btn = gr.Button("📊 Generate Report", variant="primary")

                    analytics_output = gr.Textbox(
                        label="Analytics Report",
                        lines=15,
                        show_copy_button=True
                    )

                # Module 7: Notifications
                with gr.TabItem("🔔 Notifications & Alerts"):
                    gr.Markdown("### Citizen Communication System")

                    with gr.Row():
                        with gr.Column():
                            notification_type = gr.Radio(
                                choices=["Service Update", "Emergency Alert", "Reminder", "Announcement"],
                                label="Notification Type",
                                value="Service Update"
                            )
                            target_audience = gr.CheckboxGroup(
                                choices=["All Citizens", "Specific Category", "Geographic Region", "Age Group"],
                                label="Target Audience",
                                value=["All Citizens"]
                            )
                            notification_message = gr.Textbox(
                                label="Message",
                                placeholder="Enter notification message...",
                                lines=4
                            )
                            channels = gr.CheckboxGroup(
                                choices=["SMS", "Email", "Push Notification", "Voice Call"],
                                label="Delivery Channels",
                                value=["SMS", "Push Notification"]
                            )
                            send_notification_btn = gr.Button("📤 Send Notification", variant="primary")

                        with gr.Column():
                            notification_result = gr.Textbox(
                                label="Delivery Status",
                                lines=12,
                                show_copy_button=True
                            )

                # Module 8: Authentication
                with gr.TabItem("🔐 Access Control"):
                    gr.Markdown("### Authentication & Role-based Access Management")

                    with gr.Row():
                        with gr.Column():
                            username = gr.Textbox(
                                label="Username",
                                placeholder="Enter username"
                            )
                            user_role = gr.Dropdown(
                                choices=["Citizen", "Staff", "Manager", "Admin"],
                                label="User Role",
                                value="Citizen"
                            )
                            access_level = gr.Radio(
                                choices=["Guest", "Basic", "Partial", "Full"],
                                label="Access Level",
                                value="Basic"
                            )
                            auth_btn = gr.Button("🔑 Authenticate", variant="primary")

                        with gr.Column():
                            auth_result = gr.Textbox(
                                label="Authentication Result",
                                lines=15,
                                show_copy_button=True
                            )

            # Event Handlers
            submit_query_btn.click(
                fn=self.process_citizen_query,
                inputs=[citizen_query, language_select, voice_input, query_category],
                outputs=query_response
            )

            chat_submit.click(
                fn=self.chatbot_response,
                inputs=[chat_input, chatbot],
                outputs=[chat_input, chatbot]
            )

            chat_input.submit(
                fn=self.chatbot_response,
                inputs=[chat_input, chatbot],
                outputs=[chat_input, chatbot]
            )

            verify_btn.click(
                fn=self.detect_misinformation,
                inputs=[content_text, source_url],
                outputs=verification_result
            )

            dashboard_btn.click(
                fn=self.generate_admin_dashboard,
                outputs=[query_volume_plot, category_plot, satisfaction_plot, language_plot]
            )

            load_queries_btn.click(
                fn=self.staff_query_management,
                inputs=[staff_role, filter_category, filter_status],
                outputs=[queries_table, staff_summary]
            )

            generate_report_btn.click(
                fn=self.generate_analytics_report,
                inputs=[report_type, date_range],
                outputs=analytics_output
            )

            send_notification_btn.click(
                fn=self.send_notifications,
                inputs=[notification_type, target_audience, notification_message, channels],
                outputs=notification_result
            )

            auth_btn.click(
                fn=self.authenticate_user,
                inputs=[username, user_role, access_level],
                outputs=auth_result
            )

            # Footer
            gr.Markdown(f"""
            ---
            **IBM Hackathon 2024** | Government Citizen Engagement Platform |
            Built with AI/ML, NLP, and Advanced Analytics

            **AI Status:** {'🟢 IBM Granite 3.3-2B Model Active' if self.model_loaded else '🟡 Fallback System Active'}
            """)

        return interface

def main():
    """Launch the Government Citizen Engagement Platform with IBM Granite AI"""
    print("🚀 Initializing Government Citizen Engagement Platform...")
    print("🔄 Loading IBM Granite 3.3-2B Instruct Model...")

    platform = GovernmentPlatform()
    interface = platform.create_interface()

    print("✅ Platform initialized successfully!")
    print(f"🤖 AI Model Status: {'IBM Granite 3.3-2B Active' if platform.model_loaded else 'Fallback System Active'}")

    interface.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=True,
        debug=True,
        show_error=True
    )

if __name__ == "__main__":
    main()

🚀 Initializing Government Citizen Engagement Platform...
🔄 Loading IBM Granite 3.3-2B Instruct Model...
Loading IBM Granite 3.3 2B Instruct model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-3-1f7d130f4ce8>:531: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


IBM Granite model loaded successfully!
✅ Platform initialized successfully!
🤖 AI Model Status: IBM Granite 3.3-2B Active
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2c66c51f1493411b56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
